In [55]:
import os,sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor
from keras.utils import *
from keras.utils.generic_utils import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.preprocessing.image import *
from multiprocessing import *
import gensim
from gensim.models.word2vec import *
from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
import sent2vec
import re
import string
import unicodedata as udata
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
from collections import OrderedDict
import random

In [56]:
samples_pid_train = pickle.load(open('./data/tree-rep-profiles-partial/TRAIN_pidNames.pkl','rb'))
samples_pid_valid = pickle.load(open('./data/tree-rep-profiles-partial/DEV_pidNames.pkl','rb'))
samples_pid_test = pickle.load(open('./data/tree-rep-profiles-partial/TEST_pidNames.pkl','rb'))

In [57]:
# all many2many pid
all_m2m_pid = []
root_dir = './data/tree-rep-profiles_one2many/'
rasmma_dir  = next(os.walk(root_dir))[1]
for dir_ in rasmma_dir:
    fam_dir = next(os.walk(root_dir + dir_))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + dir_+ '/' + fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + dir_ + '/' + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            all_m2m_pid.extend(hl_list)
all_m2m_pid = set(all_m2m_pid)

100%|██████████| 183/183 [00:00<00:00, 1994.31it/s]


In [58]:
api_li = ['LoadLibrary',
'CreateProcess',
'OpenProcess',
'ExitProcess',
'TerminateProcess',
'WinExec',
'CreateRemoteThread',
'CreateThread',
'CopyFile',
'CreateFile',
'DeleteFile',
'RegSetValue',
'RegCreateKey',
'RegDeleteKey',
'RegDeleteValue',
'RegQueryValue',
'RegEnumValue',
'WinHttpConnect',
'WinHttpOpen',
'WinHttpOpenRequest',
'WinHttpReadData',
'WinHttpSendRequest',
# 'WinHttpWriteData', #少了
'InternetOpen',
'InternetConnect',
'HttpSendRequest',
'GetUrlCacheEntryInfo']
# api_li = [x.lower() for x in api_li] #lowrer case?小寫?
len(api_li)

26

* 是否都轉小寫?
* sent2vec hyper parameters
* 

In [5]:
# profile = './data/tree-rep-profiles_one2many/small_short/6.allaple_0.8/G1478/c6d288841b9719a01bd340da55538ce43aaa74265791a41ee493cabde9d2402e_2960.profile'

# with open(profile,encoding='ISO 8859-1') as f:
#     lines = f.read()
# lines = re.sub(r'[^\x00-\x7F]+','', lines)
# lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
# lines.splitlines()

In [59]:
corpus = []
dil= r"[,.;\-+^()/@#?!&$:{}\\*%~\'\"\=\_]+\ *" #等號、底線被保留

processed_pid = []
root_dir = './data/tree-rep-profiles_one2many/'
rasmma_dir  = next(os.walk(root_dir))[1]
for dir_ in rasmma_dir:
    fam_dir = next(os.walk(root_dir + dir_))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + dir_+ '/' + fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + dir_ + '/' + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for hl in hl_list:
                try:
                    if hl in samples_pid_test: #移除在test裡面的hkl
                        hl_list.remove(hl)
                    if hl in samples_pid_valid: #移除在valid裡面的hkl
                        hl_list.remove(hl)
                    if hl in processed_pid:
                        hl_list.remove(hl)
                except ValueError:
                    pass
            processed_pid.extend(hl_list)
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
#             hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for profile in hl_list:
                with open(profile,encoding='ISO 8859-1') as f:
                    lines = f.read()
                lines = re.sub(r'[^\x00-\x7F]+','', lines)
                lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
                lines = lines.splitlines()
                for line in lines:
                    temp = re.sub(dil," ",line)#.lower()) #lower小寫?
                    temp = temp.split(" ")
                    temp = list(filter(None, temp))
                    temp = ' '.join(temp)
                    if temp.split(' ')[0] not in api_li:
                        print('=o2m_ERR:=',profile,'=>',temp)
                        continue
                    corpus.append(temp)

root_dir = './data/tree-rep-profiles_one2one/'
rasmma_dir  = next(os.walk(root_dir))[1]
for dir_ in rasmma_dir:
    fam_dir = next(os.walk(root_dir + dir_))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + dir_+ '/' + fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + dir_ + '/' + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for hl in hl_list:
                try:
                    if hl in all_m2m_pid: #移除已經有的pid
                        hl_list.remove(hl)
                except ValueError:
                    pass
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for profile in hl_list:
                with open(profile,encoding='ISO 8859-1') as f:
                    lines = f.read()
                lines = re.sub(r'[^\x00-\x7F]+','', lines)
                lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
                lines = lines.splitlines()
                for line in lines:
                    temp = re.sub(dil," ",line)#.lower()) #小寫?
                    temp = temp.split(" ")
                    temp = list(filter(None, temp))
                    temp = ' '.join(temp)
                    if temp.split(' ')[0] not in api_li:
                        print('=o2o_ERR:=',profile,'=>',temp)
                        continue
                    corpus.append(temp)

with open('data/sent2vec_corpus/o2o_o2m_Sent2Vec_woParam_0616.txt','w') as f:
    f.write('\n'.join(corpus))
corpus[-10:]

100%|██████████| 166/166 [00:15<00:00, 13.81it/s]


['LoadLibrary PR SYS uxtheme DLL Ret 0',
 'LoadLibrary PR SYS oleaut32 DLL Ret 0',
 'LoadLibrary PR SYS sxs DLL Ret 0',
 'LoadLibrary PR SYS oleaut32 DLL Ret 0',
 'RegQueryValue PR HKLM soft ms winNT imm PR SUBK ime file PR 0 PR 12f44c Ret 0',
 'LoadLibrary PR SYS version DLL Ret 0',
 'LoadLibrary PR SYS msctfime IME Ret 0',
 'CreateFile PR SYS IME PR GENERIC READ PR OPEN EXISTING PR FILE SHARE DELETE FILE SHARE READ Ret 0',
 'LoadLibrary PR SYS msctfime IME Ret 0',
 'CreateFile PR SYS IME PR GENERIC READ PR OPEN EXISTING PR FILE SHARE DELETE FILE SHARE READ Ret 0']

# Sent2Vec Training time 
* cd ~/Downloads/sent2vec/

## Evaluate
* TODO: 對角線相加分數要越大越好

In [60]:
with open('data/sent2vec_corpus/o2o_o2m_Sent2Vec_woParam_0616.txt','r') as f: #asp1_sentences_rep_0116
    corpus = f.read().splitlines()
corpus = sorted(corpus)
corpus[-100:]

['WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest Ret P Ret 0',
 'WinHttpSendRequest

In [61]:
model = sent2vec.Sent2vecModel()
model.load_model('model/o2o_o2m_Sent2Vec_woParam_0616.bin')
emb_all = model.embed_sentences(corpus)

In [62]:
api = ""
count = 0
count_all = {}
for row in corpus:
    try:
        count_all[row.split(' ')[0]] += 1
    except KeyError:
        count_all[row.split(' ')[0]] = 1
print(len(count_all))
sorted(count_all.items())

26


[('CopyFile', 106322),
 ('CreateFile', 763095),
 ('CreateProcess', 8546),
 ('CreateRemoteThread', 21706),
 ('CreateThread', 20157),
 ('DeleteFile', 12447),
 ('ExitProcess', 8271),
 ('GetUrlCacheEntryInfo', 42),
 ('HttpSendRequest', 1967),
 ('InternetConnect', 2037),
 ('InternetOpen', 1976),
 ('LoadLibrary', 434449),
 ('OpenProcess', 30319),
 ('RegCreateKey', 183837),
 ('RegDeleteKey', 1470),
 ('RegDeleteValue', 9931),
 ('RegEnumValue', 289918),
 ('RegQueryValue', 2119704),
 ('RegSetValue', 126745),
 ('TerminateProcess', 94),
 ('WinExec', 583),
 ('WinHttpConnect', 4461),
 ('WinHttpOpen', 4473),
 ('WinHttpOpenRequest', 4460),
 ('WinHttpReadData', 4376),
 ('WinHttpSendRequest', 4474)]

In [63]:
kk = list(count_all.keys())
for k in api_li:
    if k not in kk:
        print('少了:',k)
for k in kk:
    if k not in api_li:
        print('多了',k)

In [64]:
c= 0
df = pd.DataFrame(0, index=api_li, columns=api_li)
index_ = df.index.tolist()
avg = 100
for _ in range(avg):
    c=0
    for k,v in count_all.items():
        pick = random.randrange(c,c+v) #cosine similarity的左邊
        pick = emb_all[pick,:] #sorted corpus的embedding_all
        c1 = 0
        try:
            row = index_.index(k)
        except ValueError:
            continue
        for k1,v1 in count_all.items(): # cosine similarity的右邊
            pick1 = random.randrange(c1,c1+v1)
            pick1 = emb_all[pick1,:]
            score = cosine_similarity([pick],[pick1]) #cosine_similarity
    #         print(score)
            c1 = c1+v1
            try:
                col = index_.index(k1)
                df.iloc[row,col] += float(score)
            except ValueError:
                continue
        c = c+v
df = df/avg #similarity
# df = (df-np.min(df.values))/(np.max(df.values)-np.min(df.values)) #distance normalize
df

,LoadLibrary,CreateProcess,OpenProcess,ExitProcess,TerminateProcess,WinExec,CreateRemoteThread,CreateThread,CopyFile,CreateFile,...,RegEnumValue,WinHttpConnect,WinHttpOpen,WinHttpOpenRequest,WinHttpReadData,WinHttpSendRequest,InternetOpen,InternetConnect,HttpSendRequest,GetUrlCacheEntryInfo
LoadLibrary,0.838933,0.244649,0.514220,0.267686,0.123839,0.272965,0.040012,0.196759,0.301947,0.035850,...,-0.095165,0.189034,-0.047864,0.198881,0.257593,0.196514,0.143392,0.373013,0.010894,-0.083753
CreateProcess,0.214498,0.844274,0.352794,0.362563,0.336347,0.840449,0.330470,0.275155,0.568673,0.471193,...,0.317791,0.317055,0.221703,0.251794,0.296732,0.129591,0.395419,0.357091,0.207917,0.242502
OpenProcess,0.496648,0.320869,0.880611,0.505600,0.302287,0.378819,0.251345,0.330115,0.345681,0.152840,...,0.101204,0.242321,0.118074,0.314284,0.355643,0.229795,0.282359,0.487293,0.136560,0.010315
ExitProcess,0.234688,0.369118,0.474178,0.856750,0.551478,0.479734,0.362844,0.466335,0.287833,0.275201,...,0.165146,0.209904,0.372563,0.402813,0.242292,0.026998,0.461332,0.491646,0.030929,0.097033
TerminateProcess,0.122448,0.359952,0.318804,0.558571,0.714637,0.400053,0.575329,0.608649,0.329923,0.439238,...,0.321871,0.453099,0.361169,0.428202,0.304914,0.053774,0.459584,0.454755,0.302853,0.329193
WinExec,0.263834,0.888694,0.389857,0.500072,0.397374,1.000000,0.258979,0.253551,0.440990,0.371426,...,0.186709,0.251534,0.133485,0.330331,0.340416,0.179394,0.435169,0.517017,0.247322,0.239312
CreateRemoteThread,0.039739,0.337438,0.255487,0.365676,0.586463,0.266953,0.938268,0.759506,0.451967,0.620213,...,0.530817,0.579262,0.465638,0.413198,0.385954,0.081201,0.418419,0.278865,0.297642,0.345380
CreateThread,0.193777,0.283455,0.329227,0.441950,0.607582,0.250894,0.765531,0.908625,0.424249,0.601365,...,0.463117,0.531909,0.529011,0.406732,0.397798,0.178152,0.503898,0.328779,0.178649,0.243601
CopyFile,0.300565,0.578000,0.346535,0.296707,0.343332,0.460825,0.459425,0.431525,0.885928,0.575073,...,0.422926,0.480733,0.305632,0.192542,0.227845,0.132261,0.248161,0.158522,0.115483,0.107584
CreateFile,0.034370,0.461450,0.144586,0.281824,0.453090,0.373349,0.623710,0.591222,0.564110,0.852867,...,0.644607,0.473627,0.604835,0.330496,0.233522,0.083523,0.403679,0.053729,0.183049,0.259751


In [65]:
lib = ['LoadLibrary']
proc = ['CreateProcess', 'OpenProcess', 'ExitProcess', 'TerminateProcess', 'WinExec',
        'CreateRemoteThread', 'CreateThread']
file = ['CopyFile', 'CreateFile', 'DeleteFile']
reg = ['RegSetValue', 'RegCreateKey', 'RegDeleteKey', 'RegDeleteValue',
       'RegQueryValue', 'RegEnumValue']
net = ['WinHttpConnect', 'WinHttpOpen', 'WinHttpOpenRequest', 'WinHttpReadData', 'WinHttpSendRequest', #'WinHttpWriteData',
        'InternetOpen', 'InternetConnect', 'HttpSendRequest', 'GetUrlCacheEntryInfo']

#lower?
# lib = [x.lower() for x in lib]
# proc = [x.lower() for x in proc]
# file = [x.lower() for x in file]
# reg =[x.lower() for x in reg]
# net = [x.lower() for x in net]

# df.loc['']
x = [lib,proc,file,reg,net]
x = sum(x,[])
index_ , len(index_)

(['LoadLibrary',
  'CreateProcess',
  'OpenProcess',
  'ExitProcess',
  'TerminateProcess',
  'WinExec',
  'CreateRemoteThread',
  'CreateThread',
  'CopyFile',
  'CreateFile',
  'DeleteFile',
  'RegSetValue',
  'RegCreateKey',
  'RegDeleteKey',
  'RegDeleteValue',
  'RegQueryValue',
  'RegEnumValue',
  'WinHttpConnect',
  'WinHttpOpen',
  'WinHttpOpenRequest',
  'WinHttpReadData',
  'WinHttpSendRequest',
  'InternetOpen',
  'InternetConnect',
  'HttpSendRequest',
  'GetUrlCacheEntryInfo'],
 26)

In [66]:
up=0
down = len(index_)
down_cat = len(x)
up_cat = 0
up_cat2 = 0
lib_lib=lib_proc=lib_file=lib_reg=lib_net = 0
proc_lib=proc_proc=proc_file=proc_reg=proc_net = 0
file_lib=file_proc=file_file=file_reg=file_net = 0
reg_lib=reg_proc=reg_file=reg_reg=reg_net = 0
net_lib=net_proc=net_file=net_reg=net_net = 0
for api in index_:
    if df.loc[api].idxmax() == api:
        up+=1
    else:
        print(api)
    baseline_lib = df.loc[api,lib].mean()
    baseline_proc =  df.loc[api,proc].mean()
    baseline_file = df.loc[api,file].mean()
    baseline_reg = df.loc[api,reg].mean()
    baseline_net = df.loc[api,net].mean()
    max_value = np.max([baseline_lib,baseline_proc,baseline_file,baseline_reg,baseline_net])
    if (api in lib):
        lib_lib += baseline_lib
        lib_proc += baseline_proc
        lib_file += baseline_file
        lib_reg += baseline_reg
        lib_net += baseline_net
        if (max_value==baseline_lib):
            up_cat+=1
    elif api in proc :
        proc_lib += baseline_lib
        proc_proc += baseline_proc
        proc_file += baseline_file
        proc_reg += baseline_reg
        proc_net += baseline_net
        if max_value==baseline_proc:
            up_cat+=1
    elif api in file :
        file_lib += baseline_lib
        file_proc += baseline_proc
        file_file += baseline_file
        file_reg += baseline_reg
        file_net += baseline_net
        if max_value==baseline_file:
            up_cat+=1    
    elif api in reg:
        reg_lib += baseline_lib
        reg_proc += baseline_proc
        reg_file += baseline_file
        reg_reg += baseline_reg
        reg_net += baseline_net
        if max_value==baseline_reg:
            up_cat+=1    
    elif api in net:
        net_lib += baseline_lib
        net_proc += baseline_proc
        net_file += baseline_file
        net_reg += baseline_reg
        net_net += baseline_net
        if max_value==baseline_net:
            up_cat+=1
if np.max([lib_lib,lib_proc,lib_file,lib_reg,lib_net]) == lib_lib:
    up_cat2+=1
if np.max([proc_lib,proc_proc,proc_file,proc_reg,proc_net]) == proc_proc:
    up_cat2+=1
if np.max([file_lib,file_proc,file_file,file_reg,file_net]) == file_file:
    up_cat2+=1
if np.max([reg_lib,reg_proc,reg_file,reg_reg,reg_net]) == reg_reg:
    up_cat2+=1
if np.max([net_lib,net_proc,net_file,net_reg,net_net]) == net_net:
    up_cat2+=1
        
print('自己還原自己-ACC:',(up/down)*100,'%') #self
print('自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的):',(up_cat/down_cat)*100,'%') #micro
print('自己那類屬於自己那一類-ACC:',(up_cat2/5)*100,'%') #macro

RegQueryValue
自己還原自己-ACC: 96.15384615384616 %
自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的): 73.07692307692307 %
自己那類屬於自己那一類-ACC: 100.0 %


In [67]:
score = []
for i in range(len(df)):
    score.append(df.iloc[i,i])
print('對角線分數(越高越好):',np.mean(score))

對角線分數(越高越好): 0.8414100148356878


:~/Downloads/sent2vec$ ./fasttext sent2vec -input ~/leoqaz12_python/ANTS/seqgen_famclf/data/o2m_o2o_Sent2Vec_0521.txt -output ~/leoqaz12_python/ANTS/seqgen_famclf/model/sent2vec_o2m_o2o_0521 -minCount 0 -dim 700 -epoch 15 -lr 0.3 -wordNgrams 13 -loss ns -neg 10 -thread 35 -dropoutK 1 -minCountLabel 0 -bucket 2000000 -t 0.00001 -numCheckPoints 2
* 自己還原自己-ACC: 88.88888888888889 %
* 自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的) 81.48148148148148 %
* 自己那類屬於自己那一類-ACC 100.0 %
* 小寫的更高
***
濾掉=號、底線、轉小寫:
regqueryvalue
winhttpwritedata
* 自己還原自己-ACC: 92.5925925925926 %
* 自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的): 85.18518518518519 %
* 自己那類屬於自己那一類-ACC: 80.0 %
./fasttext sent2vec -input ~/leoqaz12_python/ANTS/seqgen_famclf/data/o2m_Sent2Vec_0521.txt -output ~/leoqaz12_python/ANTS/seqgen_famclf/model/sent2vec_o2m_0521 -minCount 0 -dim 700 -epoch 15 -lr 0.3 -wordNgrams 18 -loss ns -neg 10 -thread 35 -dropoutK 1 -minCountLabel 0 -bucket 2000000 -t 0.00001 -numCheckPoints 2
***
 ./fasttext sent2vec -input ~/leoqaz12_python/ANTS/seqgen_famclf/data/o2o_o2m_lower_woPunct_Sent2Vec_0522.txt -output ~/leoqaz12_python/ANTS/seqgen_famclf/model/sent2vec_o2o_o2m_lower_woPunct_0522 -minCount 0 -dim 700 -epoch 15 -lr 0.3 -wordNgrams 18 -loss ns -neg 10 -thread 35 -dropoutK 1 -minCountLabel 0 -bucket 2000000 -t 0.00001 -numCheckPoints 2
 * 自己還原自己-ACC: 96.15384615384616 %
 * 自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的): 88.46153846153845 %
 * 自己那類屬於自己那一類-ACC: 100.0 %
 ***
 ./fasttext sent2vec -input ~/leoqaz12_python/ANTS/seqgen_famclf/data/sent2vec_corpus/o2o_o2m_lower_woPunct_Sent2Vec_0522.txt -output ~/leoqaz12_python/ANTS/seqgen_famclf/model/sent2vec_o2o_o2m_lower_woPunct_0523 -minCount 0 -dim 700 -epoch 15 -lr 0.3 -wordNgrams 16 -loss ns -neg 10 -thread 35 -dropoutK 1 -minCountLabel 0 -bucket 2000000 -t 0.00001 -numCheckPoints 2
 * 自己還原自己-ACC: 96.15384615384616 %
* 自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的): 76.92307692307693 %
* 自己那類屬於自己那一類-ACC: 100.0 %
***
./fasttext sent2vec -input ~/leoqaz12_python/ANTS/seqgen_famclf/data/sent2vec_corpus/o2o_o2m_lower_Sent2Vec_0523.txt -output ~/leoqaz12_python/ANTS/seqgen_famclf/model/sent2vec_o2o_o2m_lower_0523 -minCount 0 -dim 700 -epoch 15 -lr 0.3 -wordNgrams 13 -loss ns -neg 10 -thread 35 -dropoutK 1 -minCountLabel 0 -bucket 2000000 -t 0.00001 -numCheckPoints 2
* 自己還原自己-ACC: 92.3076923076923 %
* 自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的): 80.76923076923077 %
* 自己那類屬於自己那一類-ACC: 100.0 %
***
o2o_o2m_Sent2Vec_lower_wParam_0616:

createprocess
regqueryvalue

* 自己還原自己-ACC: 92.3076923076923 %
* 自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的): 88.46153846153845 %
* 自己那類屬於自己那一類-ACC: 100.0 %
*  對角線分數(越高越好): 0.8331535527181739

***
./fasttext sent2vec -input ~/leoqaz12_python/ANTS/seqgen_famclf/data/sent2vec_corpus/o2o_o2m_Sent2Vec_lower_woParam_0616.txt -output ~/leoqaz12_python/ANTS/seqgen_famclf/model/o2o_o2m_Sent2Vec_lower_woParam_0616_768 -minCount 0 -dim 768 -epoch 15 -lr 0.3 -wordNgrams 16 -loss ns -neg 10 -thread 30 -dropoutK 1 -minCountLabel 0 -bucket 2000000 -t 0.00001 -numCheckPoints 2

regqueryvalue
* 自己還原自己-ACC: 96.15384615384616 %
* 自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的): 73.07692307692307 %
* 自己那類屬於自己那一類-ACC: 100.0 %
* 對角線分數(越高越好): 0.843296217550523
***
o2o_o2m_Sent2Vec_woParam_0616
* RegQueryValue
* 自己還原自己-ACC: 96.15384615384616 %
* 自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的): 73.07692307692307 %
* 自己那類屬於自己那一類-ACC: 100.0 %
* 對角線分數(越高越好): 0.8414100148356878

## 